In [ ]:
import os
import sys
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import random

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# WEIGHTS_PATH = os.path.join(MODEL_DIR, "mask_rcnn_seg_scenenn_0236.h5")
WEIGHTS_PATH = os.path.join(MODEL_DIR, "mask_rcnn_seg_scenenet_0259.h5")

DATA_DIR = "/home/orestis/data/records_2"

In [ ]:
# import sceneNN
from dataset import *

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Validation dataset
dataset_test = ObjectsDataset()
dataset_test.load_sceneNN(DATA_DIR, skip=9)
dataset_test.prepare()

## Detection

In [ ]:
class InferenceConfig(ObjectsConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
model.load_weights(WEIGHTS_PATH, by_name=True)

In [ ]:
ax = get_ax(rows=len(dataset_test.image_ids), cols=3, size=8)
for i, image_id in enumerate(dataset_test.image_ids):
    rgbd = dataset_test.load_image(image_id)
    image = rgbd[:, :, 0:3]
    depth = rgbd[:, :, 3]
    depth = depth / np.max(depth) * 255
    ax[i][0].imshow(depth.astype(np.uint8))
    ax[i][1].imshow(image.astype(np.uint8))
    r = model.detect([rgbd])[0]
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                    dataset_test.class_names, r['scores'], ax=ax[i][2])